In [1]:
from requests import get
from json import loads
from time import sleep
from typing import List

from borsdata_sdk.models.Instrument import Instrument
from borsdata_sdk.models.Market import Market
import pandas as pd
from http import HTTPStatus

from borsdata_sdk import BorsdataAPI

import borsdata_sdk

In [2]:
auth_key = 'f4e8e3c7c46c4c34ae238a1822863410'

RATE_LIMIT = 429


In [3]:
borsdata = BorsdataAPI(auth_key)

In [4]:
instruments = borsdata.get_instruments()

In [5]:
## Set up for loop 

my_list = []

for i in range(len(instruments)):
    tmp_list = [instruments[i].insId,instruments[i].name,instruments[i].countryId,instruments[i].sectorId,instruments[i].marketId,instruments[i].instrument]
    my_list.append(tmp_list)

instruments_df=pd.DataFrame(my_list,columns=['insID','Name','countryId','sectorId', 'marketId', 'instrument'])  

In [24]:
# Rename the lists to the real names
# MarketId: 1 = Large Cap, 2 = Mid Cap, 3=Small Cap, 4=First North

swe_stock_main = instruments_df[(instruments_df['countryId'] == 1) & (instruments_df['instrument'] == 0) & (instruments_df['marketId'].isin([1,2,3, 4]))]

In [25]:
swe_stock_main

,insID,Name,countryId,sectorId,marketId,instrument
0,2,AAK,1,2.0,1,0
1,3,ABB,1,5.0,1,0
2,6,Active Biotech,1,4.0,3,0
3,7,Addnode,1,6.0,2,0
4,8,Addtech,1,5.0,1,0
...,...,...,...,...,...,...
1860,2328,Solid Försäkring,1,1.0,3,0
1861,2329,Tobii Dynavox,1,6.0,2,0
1862,2330,Titania,1,1.0,4,0
1863,2331,Mestro,1,6.0,4,0


In [8]:
len(swe_stock_main)

368

# Import report

- use R12
    - How can I get 10 years of data?
    - the calls do not seem to work
- extract the data for the reports
- STACK files

In [ ]:
yearly_reports = borsdata.get_instrument_reports(6, 'year')
r12s = borsdata.get_instrument_reports(3, 'r12')

In [75]:
#extract all data through for loop
#r12[1].__dict__

insID = 14

reports_df = pd.DataFrame(columns=['insID','Report Date', 'N Shares'])
reports_df = reports_df.set_index(['Report Date', 'insID'])

for i in [2,3,14]:
    ind_comp_rep = borsdata.get_instrument_reports(i, 'r12')

    rep_list = []
    for rep in ind_comp_rep:
        tmp_rep = [i, rep.report_Date,rep.number_Of_Shares*1000]
        rep_list.append(tmp_rep)

 
    reports_tmp = pd.DataFrame(rep_list,columns=['insID','Report Date', 'N Shares'])
    reports_tmp['Report Date'] = pd.to_datetime(reports_tmp['Report Date'])
    reports_tmp = reports_tmp.set_index(['Report Date', 'insID'])
    reports_df = pd.concat([reports_df,reports_tmp], sort=True) 
    
reports_df = reports_df.sort_index() 


In [76]:
reports_df

,,N Shares
Report Date,insID,
2019-07-16,2,253728.0
2019-07-17,14,33374.0
2019-07-25,3,2132000.0
2019-10-23,3,2132000.0
2019-10-24,2,253728.0
2019-11-08,14,33374.0
2020-01-30,2,253728.0
2020-02-05,3,2132000.0
2020-02-14,14,33374.0


# TEST DAILY import prices

- Run for all Ids
- Merge with names
- Save to CSV file

In [39]:
# for loop to append stock prices. Daily Prices



price_df = pd.DataFrame(columns=['d', 'insID','h','l','c','o','v'])
price_df = price_df.set_index(['d', 'insID'])

for ins in swe_stock_main['insID'].values:
    tmp_entries = borsdata.get_instrument_stock_price(ins) 
    tmp_df = pd.DataFrame([entry.__dict__ for entry in tmp_entries])
    tmp_df['d'] = pd.to_datetime(tmp_df['d'])
    tmp_df['insID'] = ins
    tmp_df = tmp_df.set_index(['d', 'insID'])
    #price_df.append(tmp_df)
    price_df = pd.concat([price_df,tmp_df], sort=True) 
    
price_df = price_df.sort_index() 

close_price_df =  pd.DataFrame(price_df['c'])
monthly_close_price_df = close_price_df.groupby([close_price_df.index.get_level_values(1), pd.Grouper(freq='m', level=0)]).last().sort_index(level='d').swaplevel()

NameError: name 'test' is not defined

In [41]:
### Save to CSV

SAVE = 'YES'

if SAVE == 'YES':
    price_df.to_csv("SWE_PRICE.csv")

## Experiments
- Calculate things for all stocks


In [84]:
#short_interest['avg_si'] = 
#test['MA12'] = test.c.groupby(level='insID').apply(lambda x: x.rolling(window=12,min_periods=1).mean())

Int64Index([   3,    6,    7,    3,    6,    7,    3,    6,    7,    3,
            ...
               7, 2008,    3,    6,    7, 2008,    3,    6,    7, 2008],
           dtype='int64', name='insID', length=7838)